In [7]:
%load_ext jupyter_black

In [2]:
from dataclasses import dataclass
from typing import overload

import pandas as pd
import numpy as np
from numpy.typing import NDArray


In [12]:
from typing import Callable


class A:
    num = 1
    last = None

    def set_num(self, num: int):
        self.num = num

    def then(self, func: Callable[["A"], None] = None):
        if func:
            self.last = func(self)
        return self


a = A()
print(a.num)
(
    a.then(lambda x: x.set_num(4))
    .then(lambda x: print(x.num))
    .then(lambda x: x.set_num(x.num * 2000))
    .then(lambda x: print(x.num))
    .then(lambda x: x.then(lambda x: x.set_num(14)))
    .then(print)
    .then(
        "the format method is a callable, and the string is filled with self -> {0}".format
    )
)
print(a.last)
print(a.num)

1
4
8000
the format method is a callable, and the string is filled with self -> <__main__.A object at 0x7fef5d8d1060>
14


In [3]:
@dataclass(frozen=True)
class Wind:
    """
    surface wind representation
    """

    direction: float
    speed: float
    gust: float = 0.0

    def to_dict(self) -> dict[str, float]:
        return self.__dict__

    def to_numpy(self) -> np.ndarray:
        return np.asanyarray(tuple(self), dtype=np.float32)

    def to_pandas(self) -> pd.Series:
        return pd.Series(self.__dict__, dtype=np.float32)

    def __gt__(self, other: "Wind") -> bool:
        return max(self.speed, self.gust) > max(other.speed, other.gust)

    def __sub__(self, other: "Wind") -> "Wind":
        wdir, wspd, wgst = self.to_numpy() - other.to_numpy()
        wdir = (wdir + 180) % 360 - 180
        return Wind(wdir, wspd, wgst)

    def __abs__(self) -> "Wind":
        return Wind(*np.abs(tuple(self)))

    def __str__(self) -> str:
        wdir, wspd, wgst = map(int, self)
        return f"{wdir:03d}{wspd:02d}{f'G{wgst:02d}' if wgst else ''}KT"

    def __iter__(self):
        yield from self.to_dict().values()


wind1 = Wind(10, 10, 15.0)

print(
    f"""
__str__() -> {wind1}
>>> wind1.to_dict()
{wind1.to_dict()}
>>> wind1.to_dict().values()
{wind1.to_dict().values()}
>>> wind1.to_dict().keys()
{wind1.to_dict().keys()}
>>> wind1.to_numpy()
{wind1.to_numpy()}
>>> wind1.to_pandas()
{wind1.to_pandas()}
"""
)


__str__() -> 01010G15KT
>>> wind1.to_dict()
{'direction': 10, 'speed': 10, 'gust': 15.0}
>>> wind1.to_dict().values()
dict_values([10, 10, 15.0])
>>> wind1.to_dict().keys()
dict_keys(['direction', 'speed', 'gust'])
>>> wind1.to_numpy()
[10. 10. 15.]
>>> wind1.to_pandas()
direction    10.0
speed        10.0
gust         15.0
dtype: float32



In [4]:
wind2 = Wind(350, 15, 25)
delta = wind1 - wind2
delta, abs(delta)

(Wind(direction=20.0, speed=-5.0, gust=-10.0),
 Wind(direction=20.0, speed=5.0, gust=10.0))

In [5]:
abs(delta), max([wind1, wind2]), min([wind1, wind2])

(Wind(direction=20.0, speed=5.0, gust=10.0),
 Wind(direction=350, speed=15, gust=25),
 Wind(direction=10, speed=10, gust=15.0))

In [6]:
print(
    f"the diff = {abs(delta)}\nthe max = {max([wind1,wind2])}\nthe min = {min([wind1,wind2])}"
)

the diff = 02005G10KT
the max = 35015G25KT
the min = 01010G15KT


In [7]:
import numpy as np
import pandas as pd
import xarray as xr

xr, np, pd

/opt/venv/lib/python3.10/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


(<module 'xarray' from '/opt/venv/lib/python3.10/site-packages/xarray/__init__.py'>,
 <module 'numpy' from '/opt/venv/lib/python3.10/site-packages/numpy/__init__.py'>,
 <module 'pandas' from '/opt/venv/lib/python3.10/site-packages/pandas/__init__.py'>)

In [13]:
pd.DataFrame([Wind(250, 20), Wind(120, 15)])

,direction,speed,gust
0,250,20,0.0
1,120,15,0.0


In [8]:
a = np.arange(20)
a, a.reshape(5, 4)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]]))

In [9]:
import requests

url = "https://www.ncei.noaa.gov/pub/data/normals/1981-2010/products/station/AQC00914000.normals.txt"
r = requests.get(url)
r

<Response [200]>

In [10]:
def notspace(line: str):
    return not line.isspace()


newset = True
_bucket = {}
for line in filter(notspace, r.text.split("\n")):
    if line.startswith("---"):
        continue
    try:
        key, *values = filter(notspace, line.split())
        _bucket[key] = values
    except ValueError:
        print(line)
    # print(line.split())
_bucket["Monthly"], _bucket["mly-prcp-normal"]
# pd.DataFrame([_bucket]).columns

(['JAN',
  'FEB',
  'MAR',
  'APR',
  'MAY',
  'JUN',
  'JUL',
  'AUG',
  'SEP',
  'OCT',
  'NOV',
  'DEC'],
 ['2116R',
  '2022S',
  '1840S',
  '1780R',
  '1820R',
  '1306R',
  '1274R',
  '1383R',
  '1570S',
  '2003S',
  '2036R',
  '2242R'])

In [11]:
import requests

year = 2021
url = f"https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{year}.csv.gz"
r = requests.get(url)
r

<Response [200]>

In [12]:
from pathlib import Path
import gzip

file = Path.cwd().parent / "ghcnd_all.tar.gz"
with gzip.GzipFile(file) as f:
    f.read()

FileNotFoundError: [Errno 2] No such file or directory: '/home/leaver2000/afit/ghcnd_all.tar.gz'